In [ ]:
# AlexNet Code
# Importing Libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, ZeroPadding2D
from keras import backend as K
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
import cv2
import tensorflow as tf

# Dimensions of our images.
img_width, img_height = 256,256


# Input and training parameters
train_data_dir = '/kaggle/input/hne/hfolds/f1/train/'
validation_data_dir = '/kaggle/input/hne/hfolds/f1/validation/'
nb_train_samples = 2274
nb_validation_samples = 750
epochs = 50
batch_size = 12
chanDim = -1


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


# Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(256,256,3), kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation 
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(ZeroPadding2D(padding=(2, 2)))
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

# Compile Model with RMSprop optimizer and 1e-3 learning rate
opt=tf.keras.optimizers.RMSprop(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
  

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

history=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
# Plotting Network Layers
from keras.utils import plot_model
plot_model(model, to_file='model_alexnet.png', show_shapes=True, show_layer_names=True)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

with open('alexnet.json', 'w') as f:
    json.dump(str(history.history), f)
    
# Plotting Accuracy Curve
history_df = pd.DataFrame(history.history)
history_df[['accuracy', 'val_accuracy']].plot()
plt.title('Training accuracy vs Validation accuracy of AlexNet for HnE')
plt.ylabel('Accuracy')
plt.xlabel('No of epochs')

In [ ]:
# Plotting Loss Curve 
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
plt.title('Training loss vs Validation loss of AlexNet for HnE')
plt.ylabel('Loss')
plt.xlabel('No of epochs')
plt.savefig('1bFold1.eps')

In [ ]:
# Testing
test_data_dir = '/kaggle/input/hne/hfolds/f1/test'
test_data_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=1,
    class_mode="binary", 
    shuffle=False)

In [ ]:
# Saving Model weights
MODEL_FILE = "alexnet.h5"
model.save_weights(MODEL_FILE)

# Getting predictions for test images
probabilities = model.predict_generator(test_generator, 762)
print(probabilities)

In [ ]:
# Thresholding prediction probabilities at 0.5
prob=[]
print(len(probabilities))
for i in range(0, 762):
    if (probabilities[i][0] > 0.5):
        prob.append(1)
    else:
        prob.append(0)
    
print(prob)

In [ ]:
# Generating true labels
labels=[]
# Benign
for i in range(0, 516):
    labels.append(0)
    
# Malignant
for i in range(0, 246):
    labels.append(1)

In [ ]:
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np

# Print the confusion Matrix
cm = confusion_matrix(labels, prob)
print(cm)